In [ ]:
%pip install -Uq upgini catboost


input data


In [ ]:
import pandas as pd
from os.path import exists

#df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df_path = "/content/train.csv"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state = 0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace= True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,8,23,13.0
1,2013-01-01,8,25,47.0
2,2013-01-01,8,6,36.0
3,2013-01-01,2,9,28.0
4,2013-01-01,2,31,38.0


In [ ]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [ ]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]

test_features = test.drop(columns=["sales"])
test_target = test["sales"]


train_features.head()


,date,store,item
0,2013-01-01,8,23
1,2013-01-01,8,25
2,2013-01-01,8,6
3,2013-01-01,2,9
4,2013-01-01,2,31


In [ ]:
test_features.head()

,date,store,item
15255,2017-01-01,2,4
15256,2017-01-01,4,8
15257,2017-01-01,3,24
15258,2017-01-01,3,23
15259,2017-01-01,6,14


enrich features

In [ ]:
train_features = train_features.dropna()


In [ ]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher (
    search_keys= {
        "date" : SearchKey.DATE,
    },
    cv = CVType.time_series
)

enricher.fit(train_features,
             train_target,
             eval_set = [(test_features, test_target)])
#we can provide multiple search keys too

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=a2b47c95-68dc-44f3-a1ae-8bb46ba3b588
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


49 relevant feature(s) found with the search keys: ['date']


f_weather_date_weather_umap_48_b39cd0c4,0.0405,100.0000,"5.3591, 5.1445, 4.5569",Upgini,Weather & climate normals data,Premium
f_autofe_mul_150,0.0291,100.0000,"-0.8406, -0.4714, -0.7137",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_weather_date_weather_umap_31_fa6d9a99,0.0248,100.0000,"5.2992, 5.1168, 4.8131",Upgini,Weather & climate normals data,Premium
f_autofe_div_1847,0.0173,100.0000,"1.0301, 1.0153, 1.0067",Upgini,AutoFE: features from Markets data,Free
f_autofe_mul_253,0.0122,100.0000,"-0.5806, 0.7147, 0.2312",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_1207,0.0106,100.0000,"-1.083, 0.8279, -0.5121",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_1266,0.0074,100.0000,"0.9842, -0.8717, -0.5636",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_1180,0.0071,100.0000,"-0.0154, 0.015, 0.0083",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_1262,0.0056,100.0000,"-0.5183, -0.3214, -0.3083",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_mul_43,0.0055,100.0000,"-0.1978, -0.8095, 0.9892",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_mul_115,0.0047,100.0000,"-3.9348, 9.6498, -17.1878",Upgini,"AutoFE: features from Calendar data,Markets data",Free


Upgini,"AutoFE: features from Calendar data,Markets data",0.1120,27
Upgini,Weather & climate normals data,0.0658,3
Upgini,AutoFE: features from Markets data,0.0275,14
Upgini,AutoFE: features from Calendar data,0.0023,2
Upgini,AutoFE: feature from Calendar data,0.0017,1
Upgini,Calendar data,0.0012,1
Upgini,World economic indicators,0.0002,1


"Calendar data,Markets data",f_autofe_mul_228,f_events_date_week_cos1_f6a8c1fc,f_financial_date_silver_e4e33014,
"Calendar data,Markets data",f_autofe_mul_43,f_events_date_week_cos1_f6a8c1fc,f_financial_date_vix_7d_to_1y_634c77eb,mul
"Calendar data,Markets data",f_autofe_div_1052,f_events_date_week_cos1_f6a8c1fc,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,div
"Calendar data,Markets data",f_autofe_mul_87,f_events_date_week_cos2_b0a07cfc,f_financial_date_vix_7d_to_1y_634c77eb,mul
"Calendar data,Markets data",f_autofe_div_1096,f_events_date_week_cos2_b0a07cfc,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,div
"Calendar data,Markets data",f_autofe_mul_115,f_events_date_week_cos3_7525fe31,f_financial_date_silver_e4e33014,mul
"Calendar data,Markets data",f_autofe_mul_130,f_events_date_week_cos3_7525fe31,f_financial_date_vix_7d_to_1y_634c77eb,mul
Calendar data,f_autofe_div_1124,f_events_date_week_cos3_7525fe31,f_events_date_week_sin1_847b5db1,div
"Calendar data,Markets data",f_autofe_div_1139,f_events_date_week_cos3_7525fe31,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,div
"Calendar data,Markets data",f_autofe_div_1139,f_events_date_week_sin1_847b5db1,f_events_date_year_sin1_3c44bc64,
"Calendar data,Markets data",f_autofe_mul_145,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,mul


We detected 93 outliers in your sample.
Examples of outliers with maximum value of target:
71    198.0
23    189.0
84    187.0
Name: target, dtype: float64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 19000
After dropping target outliers size: 18907
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15255,52.3231,299.3051,153.1295,146.1756
Eval 1,3745,61.7009,583.5455,462.1708,121.3747


shap_value : a parameter which mathematically indicates how influential a feature is towards the prediction

In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files = False, random_state =0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set = [(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15255,52.3231,0.249833,0.159349,0.090483
1,Eval 1,3745,61.7009,0.262912,0.188898,0.074014


In [ ]:
enriched_train_features = enricher.transform(train_features, keep_input = True)
enriched_test_features = enricher.transform(test_features, keep_input = True)



[                                                            ] 0% Checking dataset...

You are trying to launch enrichment for 15255 rows, which will exceed the rest limit 7080.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 2920 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=5034c598-35ff-4023-9df4-d4c7b9af72b5
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


In [ ]:
enriched_train_features.head()

In [ ]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

In [ ]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values,enriched_preds, "SMAPE")